In [1]:
from algorithms import *

In [ ]:

import numpy as np
import pandas as pd
from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.conftest import fetch_covtype




In [ ]:

def test_solvers2():
    
    X_syn, y_syn = make_classification(n_samples=1000, n_features=2000,
                                       n_informative=1998, n_redundant=2,
                                       n_classes=2, random_state=42, class_sep=2.0)

    
    data = load_breast_cancer()
    X_real, y_real = data.data, data.target

    
    cov = fetch_covtype()
    X_cov, y_cov = cov.data, (cov.target == 2).astype(int)  
    X_cov, _, y_cov, _ = train_test_split(X_cov, y_cov, train_size=50000, stratify=y_cov, random_state=42)

    
    scaler = StandardScaler()
    X_syn = scaler.fit_transform(X_syn)
    X_real = scaler.fit_transform(X_real)
    X_cov = scaler.fit_transform(X_cov)

   
    X_syn_train, X_syn_test, y_syn_train, y_syn_test = train_test_split(X_syn, y_syn, test_size=0.3, random_state=1)
    X_real_train, X_real_test, y_real_train, y_real_test = train_test_split(X_real, y_real, test_size=0.3, random_state=1)
    X_cov_train, X_cov_test, y_cov_train, y_cov_test = train_test_split(X_cov, y_cov, test_size=0.3, random_state=1)

    datasets = [
        ("synthetic", X_syn_train, X_syn_test, y_syn_train, y_syn_test),
        ("real", X_real_train, X_real_test, y_real_train, y_real_test),
        ("covertype", X_cov_train, X_cov_test, y_cov_train, y_cov_test)
    ]

    solvers = ['cg', 'lbfgs', 'tron']
    results = {
        'solver': [],
        'dataset': [],
        'cpu_time_sec': [],
        'iterations': [],
        'objective_evals': [],
        'eval_to_iter_ratio': []
    }

    for solver_name in solvers:
        for dataset_name, X_train, X_test, y_train, y_test in datasets:
            print(f"\n=== Solver: {solver_name.upper()} - {dataset_name.upper()} Data ===")

            grid = cross_validate_C(X_train, y_train, solver=solver_name)
            best_C = grid.best_params_['C']
            print("Best C:", best_C)
            plot_train_val_error(grid, title=f"{solver_name.upper()} on {dataset_name.capitalize()} Data")
            evaluate_model(grid.best_estimator_, X_test, y_test)

            if solver_name == 'cg':
                solver = ConjugateGradientSolver(max_iter=1000, tol=1e-3)
            elif solver_name == 'lbfgs':
                solver = LBFGSSolver()
            elif solver_name == 'tron':
                solver = TrustRegionNewtonSolver()
            else:
                continue

            w, loss, duration, history, counter = solver.solve(X_train, y_train, C=best_C)
            print(f"Training completed in {duration:.2f} seconds. Final loss: {loss:.4f}")
            print(f"Total iterations: {len(history)}")
            print(f"Objective evaluations: {counter}")
            print(f"Eval-to-iter ratio: {counter / len(history):.2f}")
            plot_convergence(history, title=f"{solver_name.upper()} Convergence - {dataset_name.capitalize()} Data")

            results['solver'].append(solver_name)
            results['dataset'].append(dataset_name)
            results['cpu_time_sec'].append(duration)
            results['iterations'].append(len(history))
            results['objective_evals'].append(counter)
            results['eval_to_iter_ratio'].append(counter / len(history))

    df = pd.DataFrame(results)

    print("\n=== CPU Time Comparison (seconds) ===")
    print(df.pivot(index='solver', columns='dataset', values='cpu_time_sec'))

    print("\n=== Objective Eval / Iteration Ratio ===")
    print(df.pivot(index='solver', columns='dataset', values='eval_to_iter_ratio'))

if __name__ == "__main__":
    test_solvers2()
